Using PyTorch lightning from Multi-Key-Attention-in-Vision-Transformers which is just using PyTorch.  

Multi-Key-Attention-in-Vision-Transformer uses training vit_small_patch16_224 from scratch (no pre-trained weights) on the Food-101 dataset with the default attention block

the original notebook reduces the model to have:
10/12 attention block
a drop rate of .3
drop path rate of .1
batch size of 32
weight decay of 1e-4
is trained with a base learning rate of 1e-5
as it progresses through LR schedule defined in cell 4.

change as of 10/2025

drop rate of .5
drop path rate of .2
weight decay is .05

This cell imports libraries for deep learning, data handling, visualization, and sets random seeds for reproducibility

In [1]:
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Food101
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from timm import create_model


This cell defines image transformations for training and validation.  Training data is augmented with cropping, flipping, and color jittering, while validation data is resized and center cropped.  Both are converted to tensors and normalized using ImageNet statistics.

train_transform = transforms.Compose([transforms.RandomSizedCrop(224, scale(0.8, 1.0)), 
                                      transforms.RandomHorizontalFlip(), 
                                      transform.Colorjitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    tranforms.Normalize(mean[0.485, 0.456, 0.406])])
            
taking the above code and putting it into the ImageDataModule .setup() creates datasets using transforms train_dataloader() builds the training DataLoader val_dataloader() builds the validation Dataloader - this is auto generated in the init so it's cleaner.
Organize Datasets into a lightningDataModule in PyTorch Lightning, it's best practice to use a LightningDataModule to handle your dataset loading and transforms.  Wrapping transforms and dataloaders:

In [2]:
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
from torchvision import transforms
import pytorch_lightning as pl


# ==========================
# DataModule
# ==========================

class Food101DataModule(pl.LightningDataModule):
    def __init__(self, data_dir="./data", batch_size=32, num_workers=4):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
        ])

        self.val_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def setup(self, stage=None):
        self.train_dataset = Food101(root=self.data_dir, split="train", transform=self.train_transform, download=True)
        self.val_dataset = Food101(root=self.data_dir, split="test", transform=self.val_transform, download=True)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, 
                          batch_size=self.batch_size, 
                          shuffle=True, 
                          num_workers=self.num_workers,
                          persistent_workers=True
                          )

    def val_dataloader(self):
        return DataLoader(self.val_dataset, 
                          batch_size=self.batch_size, 
                          shuffle=False, 
                          num_workers=self.num_workers,
                          persistent_workers=True
                          )

    # Fix for Lightning 2.x compatibility
    # @property
    # def allow_zero_length_dataloader_with_multiple_devices(self):
    #     return False


create_model instantiates a Vision Transformer (small variant) from timm.
num_classes=101 sets the final linear head to output 101 logits(Food101)
Now I'm still using this ViTLayer Reduction model but putting it into lightning
This time, weight_decay is at 0.5 because of overfitting from previous code.
Also changing the drop_rate to 0.5 and drop_path_rate to 0.2 (regularization hyperparameters) - dropout and stochastic depth.

In [3]:
from timm import create_model

'''By default, PyTorch uses full float32 precision for matrix multiplications.

Tensor Cores on modern GPUs can accelerate mixed-precision math (float16) or reduced-precision float32 computations.

Trade a tiny bit of numerical precision for faster training.
'''
torch.set_float32_matmul_precision('medium')

# ==========================
# ViT model
# ==========================
class ViTLayerReduction(nn.Module):
    def __init__(self, num_blocks=10, num_classes=101):
        super().__init__()
        full_model = create_model(
            "vit_small_patch16_224",
            pretrained=False,
            num_classes=num_classes,
            drop_rate=0.5,
            drop_path_rate=0.2
        )
        self.patch_embed = full_model.patch_embed
        self.cls_token = full_model.cls_token
        self.pos_embed = full_model.pos_embed
        self.pos_drop = full_model.pos_drop
        self.blocks = nn.Sequential(*list(full_model.blocks[:num_blocks]))
        self.norm = full_model.norm
        self.head = full_model.head

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        x = self.blocks(x)
        x = self.norm(x)
        return self.head(x[:, 0])

Shape example (for 224x224, patch_size=16):
Image x: (B,3,224,224)
After patch_embed: (B, 196, D) - 14x14 = 196 patches

full_model.blocks is a list of Transformer encoder blocks (attention +MLP).
This keeps only the first 10 blocks (layer reduction).  Wrapping them in nn.Sequential allows you to call them as one module.

Why?  Fewer blocks mean fewer parameters & FLOPS, and possibly less overfitting.

norm is the final layer norm applied to token embeddings.
head is the classification head (a linear mapping D -> num_classes)

The class token is at index 0 after concatenation; x[:0] extracts it per batch entry
head returns raw logits (not softmax).  Loss functions like CrossEntropyLoss expects logits

In [4]:
import math
import time

# ==========================
# Lightning Module
# ==========================
class LitViT(pl.LightningModule):
    def __init__(self, num_classes=101, base_lr=1e-5, peak_lr=1e-4, final_lr_fraction=0.1,
                 num_epochs=60, warmup_epochs=15, rampup_epochs=15, weight_decay=0.5):
        super().__init__()
        self.save_hyperparameters()
        self.model = ViTLayerReduction(num_blocks=10, num_classes=num_classes)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        self.epoch_start_time = None

    def on_train_epoch_start(self):
        self.epoch_start_time = time.time()

    def on_train_epoch_end(self):
        duration = time.time() - self.epoch_start_time
        train_loss = self.trainer.callback_metrics.get("train_loss_epoch")
        train_acc = self.trainer.callback_metrics.get("train_acc_epoch")
        val_loss = self.trainer.callback_metrics.get("val_loss")
        val_acc = self.trainer.callback_metrics.get("val_acc")
        # Epoch number starting at 1
        epoch_num = self.current_epoch + 1
        print(f"Epoch {epoch_num} completed in {duration:.2f} seconds")
        print(f"Train Loss: {train_loss:.4f}  Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}  Val Acc: {val_acc:.4f}\n")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()

        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_step=True, on_epoch=True)

        # Log LR
        optimizer = self.optimizers()
        current_lr = optimizer.param_groups[0]['lr']
        self.log("lr", current_lr, prog_bar=True, on_step=True, on_epoch=False)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()

        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_step=False, on_epoch=True)

    # Step-level LR schedule
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.peak_lr, weight_decay=self.hparams.weight_decay)

        def lr_lambda(step):
            steps_per_epoch = self.trainer.estimated_stepping_batches / self.hparams.num_epochs
            epoch_step = step / steps_per_epoch

            if epoch_step < self.hparams.warmup_epochs:
                return self.hparams.base_lr / self.hparams.peak_lr
            elif epoch_step < self.hparams.warmup_epochs + self.hparams.rampup_epochs:
                progress = (epoch_step - self.hparams.warmup_epochs) / self.hparams.rampup_epochs
                lr = self.hparams.base_lr + progress * (self.hparams.peak_lr - self.hparams.base_lr)
                return lr / self.hparams.peak_lr
            else:
                decay_progress = (epoch_step - self.hparams.warmup_epochs - self.hparams.rampup_epochs) / max(1, self.hparams.num_epochs - self.hparams.warmup_epochs - self.hparams.rampup_epochs)
                cosine_decay = 0.5 * (1 + math.cos(math.pi * decay_progress))
                lr = self.hparams.final_lr_fraction * self.hparams.peak_lr + (1 - self.hparams.final_lr_fraction) * self.hparams.peak_lr * cosine_decay
                return lr / self.hparams.peak_lr

        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)
        return {"optimizer": optimizer, "lr_scheduler": {"scheduler": scheduler, "interval": "step"}}


Lightning wraps training boilerplate so you write only the core logic.
LitViT wraps the model and stores configuration(learning rate).  It still calls forward() for inference.  Keep it simple -- delegate to the underlying module.

training_step is called for each training batch
batch is typically (inputs, labels) from DataLoader.
Compute logits -> compute cross_entropy loss (combines log_softmax + nll_loss).
self.log records metrics; prog_bar=True shows it in the progress bar.
Returning loss tells Lightning to run backprop with it.

The validation step is similar to the training step, but for validation batches.  NO optimizer step.

Training the model now...
Training orchestrates the whole run: device placement, loop, checkingpoint (if configured), logging, etc.
accelerator="gpu", devices=1 runs on 1 GPU(if available).  Use "cpu" or remove parameters if you don't have a GPU.
trainer.fit(model, data_module) starts training.  data_module supplies train_dataloader() and val_dataloader()

In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# ==========================
# Training Script
# ==========================
if __name__ == "__main__":
    batch_size = 32
    max_epochs = 60

    data_module = Food101DataModule(batch_size=batch_size)
    model = LitViT(num_classes=101, num_epochs=max_epochs)

    checkpoint_callback = ModelCheckpoint(
        monitor="val_acc",
        mode="max",
        save_top_k=1,
        filename="best-vit-{epoch:02d}-{val_acc:.4f}"
    )

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1 if torch.cuda.is_available() else None,
        precision=16,
        callbacks=[checkpoint_callback],
        log_every_n_steps=50
    )

    trainer.fit(model, datamodule=data_module)

    print(f"Best model saved at: {checkpoint_callback.best_model_path}")

c:\Users\MILLAC24\Miniconda3\envs\pytorch_env\lib\site-packages\lightning_fabric\connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\MILLAC24\Miniconda3\envs\pytorch_env\lib\site-packages\pytorch_lightning\utilities\model_summary\model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ViTLayerReduction | 18.2 M | train
1 | criterion | CrossEntropyLoss  | 0      | train
--------------------------------------------------------
18.2 M    Tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 1 completed in 141.39 seconds
Train Loss: 4.4828  Train Acc: 0.0354
Val Loss: 4.2580  Val Acc: 0.0806



...

Epoch 59 completed in 136.82 seconds
Train Loss: 2.6922  Train Acc: 0.4371
Val Loss: 2.5143  Val Acc: 0.4817



Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 60 completed in 135.88 seconds
Train Loss: 2.6815  Train Acc: 0.4424
Val Loss: 2.5121  Val Acc: 0.4832



`Trainer.fit` stopped: `max_epochs=60` reached.


Best model saved at: c:\Users\MILLAC24\Documents\GitHub\Multi-Key-Attention-in-Vision-Transformers\lightning_logs\version_8\checkpoints\best-vit-epoch=59-val_acc=0.4832.ckpt


Shapes: ensure DataLoader returns (images, labels) with labels as integers 0..100 for Food101.
Final layer: num_classes= 101 must match dataset labels.  If you change dataset, update the head.
Pretrained weights: pretrained=Flase in create_model.  If switched to True may need to adapt or freeze layers.
Regularization: drop_rate, drop_path_rate, and weight_decay all affect overfitting; tune them one at a time.
Batch size/num_workers: set appropriately for GPU and CPU
Logging metrics: log accuracy (torchmetrics compute acc = (logits.argmax(Dim=1) == y).float().mean())
Learning rate schedulers: add ReduceLROnPlateau or CosineAnnealingLR in configure_optimizers (done inside configure_optimizers).  Use LambdaLR scheduler but normalized by epoch.  This works per-step not per-epoch.
Epoch is more stable and reduces noise.